# Integrated Slew

This notebook is used to execute the [LVV-T2215 (2.0)] test script during System Spread Integration Tests on Level 3.  
It is part of the plan [LVV-P81] and of the test cylce [LVV-C175].   
Execution steps are separated by horizontal lines.   
Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA.

History:
 - [LVV-T2215 (1.0)] has a notebook which doesn't do the slew sequence described on the test script. For this reason the version 2.0 was created.

 - [LVV-T2215 (2.0)] is similar to [LVV-T2290 (2.0)] test case, but without taking a ComCam image. 
 
Execution steps are separated by horizontal lines. Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA. 

Last executed by J. Esteves 20220408

[LVV-T2215 (1.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/2590
[LVV-T2215 (2.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2215
[LVV-T2290 (2.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2290  
[LVV-P81]: https://jira.lsstcorp.org/secure/Tests.jspa#/testPlan/LVV-P81
[LVV-C175]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCycle/LVV-C175

---
## Under Construction:


Things to do:
1. Add log info
2. Test on Tucson 


In [ ]:
from lsst.ts import utils

# Extract your name from the Jupyter Hub
__executed_by__ = os.environ["JUPYTERHUB_USER"]  

# Extract execution date
__executed_on__ = utils.astropy_time_from_tai_unix(utils.current_tai())
__executed_on__.format = "isot"

# This is used later to define where Butler stores the images
summit = os.environ["LSST_DDS_PARTITION_PREFIX"] == "summit"

print(f"\nExecuted by {__executed_by__} on {__executed_on__}."
      f"\n  At the summit? {summit}")

---
## Load all the needed libraries

Using the setup procedure, get the remotes and the components ready.

This includes simulators as well as real hardware when available (this will depend on when the test is conducted at NCSA or on level 3 or on the telescope):

- pointing  
- mount ( with the CCW)  
- rotator  
- ready M1M3: raise mirror, turn on FB, clear forces. Note that if used at level 3, we need to have M1M3 LUT use mount telemetry  
- ready M2: turn on FB, clear forces. Note that if used at level 3, we need to have M2 LUT use mount telemetry  
- Get cam hex Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Get M2 hex (simulator) Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Finally, get the MTAOS CSC ready  

***
Run the setup.ipnyb notebook to bring all components up and in their enabled position. Check Chronograph.

***

Check Chronograph.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
import os
import sys
import asyncio
import logging

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

---
## Slew Sequence

Exercise the telescope on elevation between 86.5º and 82.5º with 4 slews. <b> No tracking is needed </b>

target_1 -> az = 180.0$^o$, el = 86.5$^o$  
target_2 -> az = 180.0$^o$, el = 85.0$^o$  
target_3 -> az = 180.0$^o$, el = 83.5$^o$  
target_4 -> az = 180.0$^o$, el = 82.0$^o$  

In [ ]:
caution = False

if not caution:
    print("Slew 1: el=86.5")
    await mtcs.point_azel(az=180.0, el=86.5)

    print("Slew 2: el=85.0")
    await mtcs.point_azel(az=180.0, el=85.0)

    print("Slew 3: el=83.5")
    await mtcs.point_azel(az=180.0, el=83.5)

    print("Slew 4: el=82.0")
    await mtcs.point_azel(az=180.0, el=82.0)
    
# await mtcs.stop_tracking()

***
Slew to target 1:

In [ ]:
await mtcs.point_azel(az=180.0, el=86.5)

---
Slew to target_2:

In [ ]:
await mtcs.point_azel(az=180.0, el=85.0)

---
Slew to target_3

In [ ]:
await mtcs.point_azel(az=180.0, el=83.5)

---
Slew to target 4

In [ ]:
await mtcs.point_azel(az=180.0, el=82.0)

---
Stop tracking to prevent hitting the Rotator soft limit.

In [ ]:
await mtcs.stop_tracking()

***
## Plot The Results


In [ ]:
end = Time(datetime.now(), scale='tai')
start = end - timedelta(seconds=1000)

dfm = await client.select_time_series('lsst.sal.MTMount.elevation', '*', start, end, csc_index)
dfm1m3 = await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedElevationForces', '*', start, end, csc_index)
dfm2 = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', start, end, csc_index)
dfh = await client.select_time_series('lsst.sal.MTHexapod.application', '*', start, end, csc_index)

idx1=dfh.MTHexapodID==1
dfh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
dfh2 = dfh[idx2]

fig, ax = plt.subplots(1,1, figsize=(15,4))
plt.plot(dfm.actualPosition, '--', label='mount elevation')
plt.plot(dfm1m3.yForces0, label='M1M3 elevation y-force 101')
plt.plot(dfm2.lutGravity0, label='M2 elevation force B1')
plt.plot(dfh1.position1, label='Camera hexapod y')
plt.plot(dfh2.position1, label='M2 hexapod y')
plt.grid()
plt.legend()

***
## Wrap Up and Shut Down

This cell is not currently included as part of the test execution, but included here as needed to shutdown the systems

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
await mtcs.standby()